# 波士頓房價預測(回歸問題)

波士頓房價是一個線性回歸問題。用來預測連續值而不是離散標籤(非分類)

我們使用 1970 年代美國波士頓郊區資料，有犯罪率丶財產稅等問題。

keras.datasets 中 'boston_housing' 的資料有點少，只有 506 筆資料， 404 訓練樣本及 102 測試樣本

還有一個問題就是資料特徵值有不同單位(a 特徵值為 0-1 之間，b 特徵值為1-10000 之間)，這會造成一系列問題，我們要對資料先做 `feature scaling` 的動作。

<br>


In [1]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

57344/57026 [==============================] - 0s 1us/step


In [5]:
# 先看看資料

print("train_data.shape = ", train_data.shape)
print("test_data.shape = ", test_data.shape)
print("train_targets.shape = ", train_targets.shape)
print("test_targets.shape = ", test_targets.shape)
print("---"*20)
print("第一筆訓練資料: ", train_data[0])
print("第一筆房價(單位:$1000美元): ", train_targets[0])

train_data.shape =  (404, 13)
test_data.shape =  (102, 13)
train_targets.shape =  (404,)
test_targets.shape =  (102,)
------------------------------------------------------------
第一筆訓練資料:  [  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
第一筆房價(單位:$1000美元):  15.2


<br>

__從第一筆訓練資料看到的這些值都是甚麼 ? 不重要，都交給機器自己去判斷就好了。深度學習就是要機器自己找到特徵值與結果值之間的關係的。__

<br>


In [9]:
# 準備資料，先進行 feature scaling。

# 我們的目的是要使資料以 0 為中心，並以標準差為單位。所以要對資料做的動作就是減去特徵平均(mean)除以標準差(std)

mean = train_data.mean(axis=0) # 沿著第 0 軸算平均
train_data -= mean # 訓練資料集減掉平均值
std = train_data.std(axis=0)
train_data /= std

# 測試資料也要做。
test_data -= mean
test_data /= std

print("mean = ", mean)
print("std = ", std)

mean =  [3.74511057e+00 1.14801980e+01 1.11044307e+01 6.18811881e-02
 5.57355941e-01 6.26708168e+00 6.90106436e+01 3.74027079e+00
 9.44059406e+00 4.05898515e+02 1.84759901e+01 3.54783168e+02
 1.27408168e+01]
std =  [9.22929073e+00 2.37382770e+01 6.80287253e+00 2.40939633e-01
 1.17147847e-01 7.08908627e-01 2.79060634e+01 2.02770050e+00
 8.68758849e+00 1.66168506e+02 2.19765689e+00 9.39946015e+01
 7.24556085e+00]


<br>

__這邊 mean 與 std 要著，之後如果部屬的話，實際 input 進來的值都要進 feature scaling 這一步。__

<br>

In [10]:
# 建立神經網路

from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1], )))
    model.add(layers.Dense(64, activation='relu')) # 非線性轉換
    model.add(layers.Dense(1)) # 輸出層為純量，線性轉換
    model.compile(optimizer='rmsprop', loss="mse", metrics=["mae"]) # 下方做說明
    return model

<br>

* Dense(1) : 純量回歸會輸出一個浮點數數值，也就是回歸值。結束時沒有使用激勵函數(e.g 'relu')代表這是一個線性層

* 結尾處不加任何 activation func 表示對輸出結果值範圍不加以任何限制。純線性輸出。

* 使用 __mse__(mean square error 均方差) 做損失函數，mse 會計算預測值與目標值間的差異平方。回歸問題常用 loss func。

* 評量指標用 __mae__(mean absolute error 平均絕對誤差)。這是預測值於實際值差異絕對值。 e.g mae = 0.5 代表預測與實際差距 500 美金。

<br>

## K 折 (K-fold) 驗證

<br>

k 折交叉驗證，用在訓練資料集過小的情況。驗證分數會因為驗證資料集切割不同而產生很大的 __變異性__。使用 K 折驗證可以有效避免這種問題。

簡單說，就是把資料切割成 N 份，在 Model 保持不變情況下，拿第一份當作驗證資料，其他做訓練資料，跑過一遍之後，換第二份當驗證資料，再跑一遍。依此類推直到每一份資料都被拿來當作驗證資料使用過。最終驗證分數是所有驗證次數的分數做平均。

以下為實現過程。

<br>

In [16]:
import numpy as np

K = 4 # 4 折交叉驗證

num_val_samples = len(train_data) // K #每一份的資料集大小

num_epochs = 100

all_scores = []

for i in range(K):
    print("執行第 ", i+1, " 折...")
    
    val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]
    
    partical_train_data = np.concatenate(
        [train_data[ : i * num_val_samples],
        train_data[( i + 1 ) * num_val_samples : ]],
    axis=0)
    
    partical_train_targets = np.concatenate(
        [train_targets[ : i * num_val_samples],
        train_targets[( i + 1 ) * num_val_samples : ]],
    axis=0)
    
    model = build_model()
    model.fit(partical_train_data, partical_train_targets, epochs=num_epochs, batch_size=1, verbose=0)  # verbose=0 靜音模式
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

執行第  1  折...
執行第  2  折...
執行第  3  折...
執行第  4  折...


In [18]:
print("all_scores = ", all_scores)
print("平均差: ", np.mean(all_scores))

all_scores =  [2.195553779602051, 2.816535234451294, 2.734400987625122, 2.39353346824646]
平均差:  2.5350058674812317


<br>

結果發現誤差值約 2530 美金左右。結果非常不理想。可以嘗試調整 Model 的神經元個數來調整。

當資料很少時，盡量使用淺層神經網路來訓練，以防止過度擬合現象發生。

<br>